In [ ]:
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import random
import numpy as np

n=0
N=5
star=[]

t=np.linspace(0,1,100)
while n<=N:
    star=[]
    rt=np.array([random.random(),random.random(),random.random()])
    star=star+[rt,random.random()]
    print(star)
    n+=1

    for i in range (n,N):
        for j in range(n,N):
            if i!=j:
                def gravity(state,t):
                    rt[0],rt[1],rt[2],vx,vy,vz=state
                    
                

            


    


[array([0.67562629, 0.91773959, 0.69416086]), 0.6863780695856248]
[array([0.37120322, 0.05907876, 0.62002148]), 0.07126628735520635]
[array([0.1226143 , 0.40500684, 0.30450936]), 0.8827966710528116]
[array([0.51277819, 0.532594  , 0.602617  ]), 0.0722719881861682]
[array([0.73875164, 0.96152276, 0.95285029]), 0.23467372048143242]
[array([0.39155286, 0.848426  , 0.80413438]), 0.16913291263930175]
